In [39]:
import pandas as pd
from scipy.stats import nbinom
import numpy as np
import matplotlib.pyplot as plt

In [40]:
df = pd.read_csv('08_09_season.csv') # upload data using 08_09_season.csv, found on github repository
df.head(10)
#df.describe()

,team_id_home,home_team,game_id,game_date,matchup_home,wl_home,fgm_home,fga_home,fg_pct_home,fg3m_home,...,fgm_away,fga_away,fg_pct_away,fg3m_away,fg3a_away,fg3_pct_away,ftm_away,fta_away,ft_pct_away,pts_away
0,1610612738,BOS,20800001,28/10/2008 00:00,BOS vs. CLE,W,33,74,0.446,3,...,29,68,0.426,3,15,0.200,24,33,0.727,85
1,1610612747,LAL,20800003,28/10/2008 00:00,LAL vs. POR,W,37,79,0.468,7,...,29,84,0.345,10,24,0.417,8,14,0.571,76
2,1610612741,CHI,20800002,28/10/2008 00:00,CHI vs. MIL,W,35,69,0.507,5,...,37,78,0.474,6,14,0.429,15,20,0.750,95
3,1610612752,NYK,20800007,29/10/2008 00:00,NYK vs. MIA,W,42,87,0.483,9,...,42,92,0.457,6,23,0.261,25,33,0.758,115
4,1610612753,ORL,20800004,29/10/2008 00:00,ORL vs. ATL,L,32,87,0.368,4,...,38,86,0.442,5,20,0.250,18,31,0.581,99
5,1610612746,LAC,20800015,29/10/2008 00:00,LAC vs. LAL,L,30,78,0.385,5,...,41,80,0.513,10,19,0.526,25,33,0.758,117
6,1610612744,GSW,20800014,29/10/2008 00:00,GSW vs. NOH,L,36,82,0.439,9,...,41,82,0.500,7,17,0.412,19,27,0.704,108
7,1610612764,WAS,20800006,29/10/2008 00:00,WAS vs. NJN,L,30,81,0.370,5,...,37,78,0.474,6,19,0.316,15,18,0.833,95
8,1610612750,MIN,20800009,29/10/2008 00:00,MIN vs. SAC,W,42,87,0.483,3,...,36,80,0.450,5,17,0.294,19,25,0.760,96
9,1610612762,UTA,20800013,29/10/2008 00:00,UTA vs. DEN,W,37,79,0.468,2,...,36,77,0.468,4,13,0.308,18,28,0.643,94


In [41]:
df['matchup_home'].str.split(' vs ').str[0] #

0       BOS vs. CLE
1       LAL vs. POR
2       CHI vs. MIL
3       NYK vs. MIA
4       ORL vs. ATL
           ...     
1225    BOS vs. WAS
1226    CLE vs. PHI
1227    NYK vs. NJN
1228    MEM vs. ATL
1229    MIN vs. SAC
Name: matchup_home, Length: 1230, dtype: object

In [42]:
for i, row in df.iterrows(): # create column with away team
    df.at[i, 'away_team'] = row['matchup_home'].split()[2]

In [43]:
# refine dataframe, derive 2 point field goals and column with 1 if home team won
dfn = df[['game_id', 'game_date', 'home_team', 'away_team', 'fg3m_home', 'ftm_home', 'pts_home', 'fg3m_away', 'ftm_away', 'pts_away']]
dfn['fg2m_home'] = df['fgm_home'] - dfn['fg3m_home']
dfn['fg2m_away'] = df['fgm_away'] - dfn['fg3m_away']
dfn['home_win'] = (dfn['pts_home'] > dfn['pts_away']).astype(int)
# dfn['home_code'] = dfn['home_team'].apply(lambda x: hash(x) % (10**8))

C:\Users\2018l\AppData\Local\Temp\ipykernel_16288\4130707652.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfn['fg2m_home'] = df['fgm_home'] - dfn['fg3m_home']
C:\Users\2018l\AppData\Local\Temp\ipykernel_16288\4130707652.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfn['fg2m_away'] = df['fgm_away'] - dfn['fg3m_away']
C:\Users\2018l\AppData\Local\Temp\ipykernel_16288\4130707652.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

In [44]:
#assign codes for teams alphabetically

team_code_map = {team: idx for idx, team in enumerate(sorted(dfn['home_team'].unique()))}
dfn['home_code'] = dfn['home_team'].map(team_code_map)
dfn['away_code'] = dfn['away_team'].map(team_code_map)

C:\Users\2018l\AppData\Local\Temp\ipykernel_16288\3788216166.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfn['home_code'] = dfn['home_team'].map(team_code_map)
C:\Users\2018l\AppData\Local\Temp\ipykernel_16288\3788216166.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfn['away_code'] = dfn['away_team'].map(team_code_map)


In [45]:
dfn.head(5)

,game_id,game_date,home_team,away_team,fg3m_home,ftm_home,pts_home,fg3m_away,ftm_away,pts_away,fg2m_home,fg2m_away,home_win,home_code,away_code
0,20800001,28/10/2008 00:00,BOS,CLE,3,21,90,3,24,85,30,26,1,0,16
1,20800003,28/10/2008 00:00,LAL,POR,7,15,96,10,8,76,30,19,1,1,19
2,20800002,28/10/2008 00:00,CHI,MIL,5,33,108,6,15,95,30,31,1,2,26
3,20800007,29/10/2008 00:00,NYK,MIA,9,27,120,6,25,115,33,36,1,3,18
4,20800004,29/10/2008 00:00,ORL,ATL,4,17,85,5,18,99,28,33,0,4,27


In [46]:
dfn.to_csv('data.csv', index=False) # save dataframe as .csv